importing packages

In [40]:
import pandas as pd
import numpy as np
import os
import getpass
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
import gradio as gr


In [ ]:
# data_directory='E://Electro Pi//courses//archive2'
# data=[]
# for folder in os.listdir(data_directory):
#     for subfolder in os.listdir(f'{data_directory}/{folder}'):
#             # print(folder)
#             # print(subfolder)
#             file = open(f"{data_directory}/{folder}/{subfolder}", "r",encoding="utf8")
#             content = file.read()
#             #print(type(content))
#             data.append({'Folder': folder, 'Subfolder': subfolder, 'Content': content})

#             file.close()
# df = pd.DataFrame(data)

In [ ]:
df

,Folder,Subfolder,Content
0,Culture,0000.txt,ينظم معهد الشارقة للفنون معرضاً فنياً تحت عنوا...
1,Culture,0001.txt,تقيم الفنانة ليتا كابيلوت معرضاً في مطلع العام...
2,Culture,0002.txt,تتواصل الليلة مسيرة التحدّي والمتعة والإثارة ض...
3,Culture,0003.txt,عقدت ظهر أمس الأول في قصر الثقافة في الشارقة ا...
4,Culture,0004.txt,اختارت صحيفة «التايمز» جورج أورويل ليحل في الم...
...,...,...,...
45495,Tech,6495.txt,مع إسدال الستار على معرض جيتكس 2013 الدولي الم...
45496,Tech,6496.txt,أبوظبي: «الخليج» حصل محمد نصر عابدين الرئيس ال...
45497,Tech,6497.txt,"برشلونة - ""الخليج"":اختتمت الهيئة العامة لتنظيم..."
45498,Tech,6498.txt,وثقت مركاتور، ذراع تكنولوجيا المعلومات في مجمو...


In [5]:
#df.to_csv('E://Electro Pi//courses//data-task02.csv',encoding='utf-8')
df=pd.read_csv('/content/data-task02.csv',encoding='utf-8')

work on 80% of each group ('Type')

In [6]:
# Define a function to select the first 80% of each group
def select_first_80_percent(group):
    n_rows = int(len(group) * 0.8)  # Calculate 80% of the rows
    return group.iloc[:n_rows]  # Select the first 80%

# Apply this function to each group in the 'Folder' column
sampled_df = df.groupby('Folder', group_keys=False).apply(select_first_80_percent)
sampled_df

,Unnamed: 0,Folder,Subfolder,Content
0,0,Culture,0000.txt,ينظم معهد الشارقة للفنون معرضاً فنياً تحت عنوا...
1,1,Culture,0001.txt,تقيم الفنانة ليتا كابيلوت معرضاً في مطلع العام...
2,2,Culture,0002.txt,تتواصل الليلة مسيرة التحدّي والمتعة والإثارة ض...
3,3,Culture,0003.txt,عقدت ظهر أمس الأول في قصر الثقافة في الشارقة ا...
4,4,Culture,0004.txt,اختارت صحيفة «التايمز» جورج أورويل ليحل في الم...
...,...,...,...,...
44195,44195,Tech,5195.txt,في دراسة استطلاعية أعدتها المؤسسة البحثية والا...
44196,44196,Tech,5196.txt,أعلنت أمس أوبتيموس للتقنية والاتصالات، الموزع ...
44197,44197,Tech,5197.txt,تتضمن مشاركة دو في أسبوع جيتكس للتقنية هذا الع...
44198,44198,Tech,5198.txt,في وقت تصل أسواق الاتصالات الخليجية حد الإشباع...


In [7]:
documents = [Document(page_content=text) for text in sampled_df['Content']]

splitting data into documents

In [8]:
text_splitter =RecursiveCharacterTextSplitter(chunk_size=30000,chunk_overlap=3000,add_start_index=True)

In [9]:
all_splits = text_splitter.split_documents(documents)
#len(all_splits)

In [ ]:
for i in range(len(all_splits)):
    print(i , len(all_splits[i].page_content))

Embedding to find similarity between texts and find relevant

In [10]:
class embedding:
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
    def embed_documents(self,docs):
        embeddings= self.model.encode(docs)
        return embeddings.tolist()
    def embed_query(self,query):
        return self.model.encode(query).tolist()

In [11]:
embed_model=embedding()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
result=embed_model.embed_query("هل ينظم معهد الشارقة للفنون معرضاً فنياً ؟")

please find my 'vector_database' here ;):
- https://drive.google.com/file/d/1PbZvQv9aHkv_4H9hwgWlxhHAFCX_rjIM/view?usp=drive_link

In [13]:
vector_database=Chroma.from_documents(documents=all_splits,embedding=embed_model)

In [14]:
retriever=vector_database.as_retriever(search_type="similarity",search_kwargs={'k':3})

In [15]:
retrieved_docs=retriever.invoke("هل ينظم معهد الشارقة للفنون معرضاً فنياً ؟")

In [16]:
retrieved_docs[0].page_content

'الشارقة - محمد ولد محمد سالم:احتضن ملتقى الأدب مساء أمس الأول ندوة بعنوان "صناعة النشر المحلية بين الواقع والتحديات" اشترك فيها كل من علي الشعالي مدير دار الهدهد، والدكتور غياث مكتبي مدير دار مكتبي للنشر والتوزيع، ومحمد باسل الحافظ مدير دار الحافظ للنشر والتوزيع، وأدارتها الدكتورة مريم الشناصي مديرة دار الياسمين للنشر والتوزيع،في مقدمتها قالت الدكتورة مريم الشناصي: إن وضع الناشر الإماراتي يحظى بدعم رسمي كبير، وإن هناك نهضة حقيقية في مجال النشر، تتمثل في وجود 100 دار نشر وتبلغ قيمة سوقها مليار درهم، 60% منها للكتاب الأجنبي، لكن الناشر ما زال يتحمل أعباء كثيرة تثقل كاهله، نظراً لغلاء صناعة الكتاب، وكثرة التكاليف التي ينبغي أن يتحملها .الدكتور غياث مكتبي قال إن الرعاية التي يحظى بها الناشر في الإمارات لا تضاهيها في الدول العربية أي رعاية أخرى، فالهيئات الرسمية تقدم تسهيلات كثيرة، وصندوق خليفة يوفر الدعم المادي للمبادرات النشرية، كما تنظم المؤسسات الرسمية ورشاً وتدريبات في مجالات عدة متعلقة بصناعة الكتاب، والبرنامج المهني الذي ينظمه معرض الشارقة للكتاب لمصلحة الناشرين والذي قدم على مدى ال

In [17]:
retrieved_docs[1].page_content

'الشارقة - "الخليج":انطلق مساء أمس الأول في مركز الأمير عبدالمحسن بن جلوي للبحوث والدراسات الإسلامية في الشارقة، مؤتمر "تحيين المعرفة، وتأصيل الإنسان" الذي ينظمه المركز على مدار يومين، بمشاركة نخبة من الباحثين والأكاديميين المتخصصين في اللغة والدراسات الإسلامية والتراثية .ويأتي المؤتمر ضمن فعاليات الشارقة عاصمة للثقافة الإسلامية، ليتوقف عند واحدة من القضايا الشائكة التي لم يتوقف الباحثون والدارسون عن طرحها، والمتعلقة بالمعارف التي قدمها التراث العربي الإسلامي، ومعالجة سائر الحضارات لها، في ظل بروز مفهوم صراع الحضارات .ترأس جلسة اليوم الأول للمؤتمر د . عمر عبدالعزيز رئيس إدارة البحوث والنشر في دائرة الثقافة والإعلام في الشارقة، بمشاركة كل من د . علي بن إبراهيم النملة، ود . حسن الأمراني، ود . عبد المجيد النجار، ود . المبروك الشيباني المنصوري، مقدمين ورقات بحثية عن "تحيين المعرفة، رؤية معاصرة وتأصيل قيمي" .وقدم د . علي بن إبراهيم النملة ورقة بعنوان: "صدام الثنائيات: افتعال الصراع بين ذاك الحين (التراث) وهذا الحين (المعاصرة)" قال فيها: "إن التراث هو الأساس الذي بنيت عليه الحضارات المتعاقبة

In [18]:
retrieved_docs[2].page_content

'الشارقة - محمد ولد محمد سالم:نظمت دائرة الثقافة والإعلام في الشارقة مساء أمس الأول في النادي الثقافي العربي في الشارقة ندوة تحت عنوان "استعادة ابن خلدون" تحدث فيها كل من الدكتور محمد صالح اللهيبي، والدكتور محمد مؤنس، وفي تقديمه للأمسية قال الدكتور عمر عبدالعزيز إن استعادة المؤرخ العربي ومؤسس علم الاجتماع عبدالرحمن بن خلدون في ندوة من هذا النوع هي محاولة لإعادة التفكير حول الأسس العقلية والعلمية في التراث، وما قدمه مفكرو الحضارة العربية من إنجازات من أجل الاستفادة من طرائق تفكيرهم وأطروحاتهم لإقامة بُنى فكرية عربية معاصرة وأصيلة، يمكن أن تسهم في تقدم الحضارة العربية والإنسانية .د . اللهيبي قال في مداخلته إن عبدالرحمن بن خلدون هو صاحب "كتاب العِبر وديوان المبتدأ والخبر في أيام العرب والعجم والبربر ومن عاصرهم من ذوي السلطان الأكبر"، الذي كان علامة فارقة في مسيرة كتابة التاريخ، وخاصة مقدمته التي صارت أساساً للتفكير التاريخي وللعلوم الإنسانية، خاصة علم الاجتماع، وأضاف اللهيبي أن ابن خلدون أعطى نظرة جديدة للتاريخ، وقدم معايير للحكم والنظر العلمي تختلف عمّا كان عليه الحال عند سابقيه، فقد نقل

construct template

In [19]:
template="""
you are an AI powered QA Assistant to provide accurate, contextually relevant answers to customer questions.
at the end of the answer you have to thank the user.
the answer should be in detail , in arabic and not less than 100 words.

if you asked about yourself , tell ask Eng. Raheem Ahmed.

what to do if answer is not included in the prompt for context:

1. apologize to user
2.tell user that you dont know the answer

for answer:
1. the output must be the answer only without any additional thoughts

knowledge you know:
{context}
Question: {question}

answer:
"""

In [20]:
custom_rag_prompt=PromptTemplate.from_template(template)

In [21]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [23]:
# getpass : used to make the api key secret and not public


os.environ["GOOGLE_API_KEY"] = getpass.getpass()



llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.2)

··········


In [25]:
rag_chain =(
    {"context":retriever | format_docs , 'question':RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [26]:
answer=rag_chain.invoke("هل ينظم معهد الشارقة للفنون معرضاً فنياً ؟")

In [27]:
print(answer)

أنا آسف، لكن لا يوجد معلومات في النص المقدم حول ما إذا كان معهد الشارقة للفنون ينظم معرضًا فنيًا أم لا. 



In [28]:
for chunk in rag_chain.stream("هل ينظم معهد الشارقة للفنون معرضاً فنياً ؟"):
    print(chunk,end="",flush=True)

أنا آسف، لكن لا يوجد معلومات في النص المقدم حول ما إذا كان معهد الشارقة للفنون ينظم معرضًا فنيًا أم لا. 


In [29]:
answer=rag_chain.invoke("هل تعلم ما اسمي؟ ؟")

In [30]:
print(answer)

أنا آسف، لكن لا أعرف اسمك. 



In [31]:
answer=rag_chain.invoke("هل ميسي كسب كاس العالم؟")
print(answer)

أنا آسف، لكنني لا أعرف إجابة هذا السؤال. 



In [57]:
import gradio as gr

def getting_answers(question, history=[]):
    answer = rag_chain.invoke(f"{question}")
    #print(answer)
    history.append((question, answer))
    retrieved_docs=retriever.invoke(f"{question}")



    return history,history ,[retrieved_docs[i].page_content for i in range(len(retrieved_docs)) ]

demo = gr.Interface(
    fn=getting_answers,
    inputs=["text", "state"],  # "text" for user input, "state" to keep track of the chat history
    outputs=["chatbot", "state","text"],  # "chatbot" for chat UI, "state" to store chat history
)

demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://91e4b93d44886b192d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [58]:
answer = rag_chain.invoke( "هل يتم  ننظيم معهد في الشارقة  ؟" )
print(answer)

نعم، ينظم معهد الشارقة للفنون معرض فنون مضيئة عند السابعة من مساء اليوم ويضم إبداعات 35 فناناً من منتسبي الدراسة في المعرض. 

أتمنى أن يكون هذا مفيدًا لك. 



In [50]:
retrieved_docs=retriever.invoke("هل ينظم معهد الشارقة للفنون معرضاً فنياً ؟")
for i in range(len(retrieved_docs)):
  print(retrieved_docs[i].page_content)

الشارقة - محمد ولد محمد سالم:احتضن ملتقى الأدب مساء أمس الأول ندوة بعنوان "صناعة النشر المحلية بين الواقع والتحديات" اشترك فيها كل من علي الشعالي مدير دار الهدهد، والدكتور غياث مكتبي مدير دار مكتبي للنشر والتوزيع، ومحمد باسل الحافظ مدير دار الحافظ للنشر والتوزيع، وأدارتها الدكتورة مريم الشناصي مديرة دار الياسمين للنشر والتوزيع،في مقدمتها قالت الدكتورة مريم الشناصي: إن وضع الناشر الإماراتي يحظى بدعم رسمي كبير، وإن هناك نهضة حقيقية في مجال النشر، تتمثل في وجود 100 دار نشر وتبلغ قيمة سوقها مليار درهم، 60% منها للكتاب الأجنبي، لكن الناشر ما زال يتحمل أعباء كثيرة تثقل كاهله، نظراً لغلاء صناعة الكتاب، وكثرة التكاليف التي ينبغي أن يتحملها .الدكتور غياث مكتبي قال إن الرعاية التي يحظى بها الناشر في الإمارات لا تضاهيها في الدول العربية أي رعاية أخرى، فالهيئات الرسمية تقدم تسهيلات كثيرة، وصندوق خليفة يوفر الدعم المادي للمبادرات النشرية، كما تنظم المؤسسات الرسمية ورشاً وتدريبات في مجالات عدة متعلقة بصناعة الكتاب، والبرنامج المهني الذي ينظمه معرض الشارقة للكتاب لمصلحة الناشرين والذي قدم على مدى الس